## Lab 04 JOINing Multiple Tables

Once you have created the physical model, generated the SQL code, and checked that the CSV files load.  Write responses to the following queries.  Note that we are connecting to the lab04 database (not 'postgres' database).

In [23]:
%load_ext sql
%sql postgres://isdb@localhost/lab04

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: isdb@lab04'

 
#### [Q01] Show the average grade and number of courses taken by those students with an avg grade > 2.5. ORDER BY student_id.

The response requires only 1 tables --- Enrollments

In [24]:
%%sql 
SELECT student_id, count(section_id),avg(grade)
FROM Enrollments 
GROUP BY student_id
HAVING avg(grade) > 2.5
ORDER BY student_id





 * postgres://isdb@localhost/lab04
7 rows affected.


student_id,count,avg
148,1,3.0000000000000000
298,2,3.0000000000000000
349,1,4.0000000000000000
358,1,4.0000000000000000
558,1,3.0000000000000000
649,2,4.0000000000000000
654,1,4.0000000000000000


#### [Q02] Produce an alphabetized list OF student NAME, course#, AND grade received by each student in ALL the courses they took. 

Now we need two tables **`Students`** and **`Enrollments`**.  To the model that we already have (with only `Enrollments`) we add `Students`

In [25]:
%%sql 
SELECT s.name, e.course_id, e.grade
FROM Enrollments as e
    JOIN Students as s on e.student_id = s.student_id
GROUP BY e.student_id, s.name, e.course_id, e.grade
ORDER BY s.name
    






 * postgres://isdb@localhost/lab04
18 rows affected.


name,course_id,grade
Allen Thomas,290,2
Allen Thomas,450,3
Bill Jones,480,2
Bob Dawson,450,3
Bob Dawson,730,1
Carol Dean,450,2
Carol Dean,480,0
Carol Dean,730,3
Howard Mansfield,290,3
Howard Mansfield,480,3


#### [Q03] How many students of each gender are enrolled in Calculus IV? 

Now we are on a roll...  The query needs three tables.  The only new tables is `Courses`

**`Students + Enrollments + Courses`**

In [26]:
%%sql
SELECT s.gender, count(s.student_id)
FROM Enrollments as e
JOIN Students as s on e.student_id = s.student_id
JOIN Courses as c on e.course_id = c.course_id
WHERE c.title = 'Calculus IV'
GROUP BY s.gender





 * postgres://isdb@localhost/lab04
2 rows affected.


gender,count
F,4
M,2


**Why stop now when we are having all this fun ... lets go for the whole enchilada ... join all five tables!**

#### [Q04]  Produce a roster for each course by instructor, listing the course title, sections they teach along with the names of all students enrolled in EACH. ORDER the roster BY teacher name.

**`Instructors + Sections + Courses + Enrollments + Students`**

If this query works correctly then your model is correct.  Give yourself a pat on th back :-)

In [27]:
%%sql
SELECT i.name, c.title, s.section_id, st.name
FROM Enrollments as e
    JOIN Students as st on e.student_id = st.student_id
    JOIN Sections as s on e.section_id = s.section_id 
    and e.course_id = s.course_id
    JOIN Courses as c on e.course_id = c.course_id
    JOIN Instructors as i on s.instructor_id = i.instructor_id
ORDER BY i.name







 * postgres://isdb@localhost/lab04
18 rows affected.


name,title,section_id,name_1
Dr. Cooke,Compiler Writing,1,John Anderson
Dr. Cooke,Compiler Writing,1,Bill Jones
Dr. Cooke,Compiler Writing,1,Janet Ladd
Dr. Engle,English Composition,1,Howard Mansfield
Dr. Engle,English Composition,1,Joe Adams
Dr. Engle,English Composition,1,Allen Thomas
Dr. Horn,Western Civilization,1,Janet Thomas
Dr. Horn,Western Civilization,1,Carol Dean
Dr. Lowe,Calculus IV,1,John Anderson
Dr. Lowe,Calculus IV,1,Bob Dawson


## ANY, ALL, EXISTS

In the following three queries we look at the use of three SQL constructs `ANY`, `ALL`, and `EXISTS`.  These constructs can be used to produce more succinct queries which other wise may have required a temporary table.

Search on the web for documentation about these constructs.  


**[Q 05] Which students received Bs in any course?**

Traditionally we would have used `IN`

In [28]:
%%sql

SELECT s.name
  FROM Students as s
 WHERE student_id IN
      (SELECT student_id 
         FROM Enrollments as e
        WHERE e.grade = 3)

 * postgres://isdb@localhost/lab04
6 rows affected.


name
Susan Powell
Bob Dawson
Howard Mansfield
Carol Dean
Allen Thomas
Val Shipp


Redo this query using `ANY`.  It requires only a small change.  But recall that the comparison operaotr before the `ANY` can be any relational operator. Compare your answer using `ANY` with that above using `IN`.

In [29]:
%%sql
SELECT s.name
  FROM Students as s
 WHERE student_id = ANY
      (SELECT student_id 
         FROM Enrollments as e
        WHERE e.grade = 3)







 * postgres://isdb@localhost/lab04
6 rows affected.


name
Susan Powell
Bob Dawson
Howard Mansfield
Carol Dean
Allen Thomas
Val Shipp


**[Q 06] Which courses have only male students?**

Produce a list of genders for all courses and check to see if 'M' is equal to all of the genders

In [33]:
%%sql
SELECT c.title
FROM Courses as c
WHERE 'M' = ALL(SELECT gender
                    FROM Enrollments as e 
                    JOIN Courses as c1 on e.course_id = c1.course_id 
                    JOIN Students as s on e.student_id = s.student_id
                     WHERE c.course_id = c1.course_id)







 * postgres://isdb@localhost/lab04
2 rows affected.


title
English Composition
Art History


**[Q 07] List all students whose average grade is greater than the lowest average grade i.e., list everyone but the person with the lowest average grade.**

Based on our approach to SQL queries so far, we would calculate the lowest average grade across all students and use that to filter the students.  But to calculate the lowest average grade across all students we need to (1) calculate the average grade for all students in one query, and stash the result in a temporary table and (2) from the temporary table determine the lowest average grade (which happens to be 1.666) (3) filter the students table.

Using the ANY  operator we can avoid a temporary table.

We only need to determine (1) the average grade for all students and then filter the students table so that the average grade of a student is greater an ANY of the values in the table from 1

In [34]:
%%sql

-- Our approach so far ...

DROP VIEW IF EXISTS V1;

CREATE VIEW V1 AS
    SELECT avg(e.grade)
      FROM Students as s
           JOIN Enrollments as e ON s.student_id = e.student_id
  GROUP BY s.name;

  SELECT s.name, avg(e.grade)
    FROM Students as s
    JOIN Enrollments as e ON s.student_id = e.student_id
   GROUP BY s.name
  HAVING avg(e.grade) > (SELECT min(avg) FROM V1)
   ORDER BY s.name ASC



 * postgres://isdb@localhost/lab04
Done.
Done.
11 rows affected.


name,avg
Allen Thomas,2.5000000000000000
Bill Jones,2.0000000000000000
Bob Dawson,2.0000000000000000
Howard Mansfield,3.0000000000000000
Janet Ladd,4.0000000000000000
Janet Thomas,4.0000000000000000
Joe Adams,4.0000000000000000
John Anderson,4.0000000000000000
Susan Powell,3.0000000000000000
Susan Pugh,2.0000000000000000


In [35]:
%%sql

-- we want to filter in all students whose avg grade is greater
-- than any grade in the below table

SELECT avg(grade)
        FROM Enrollments as e2 
       GROUP BY e2.student_id       
        

 * postgres://isdb@localhost/lab04
12 rows affected.


avg
1.6666666666666667
2.0000000000000000
4.0000000000000000
2.0000000000000000
4.0000000000000000
3.0000000000000000
3.0000000000000000
2.0000000000000000
4.0000000000000000
2.5000000000000000


The improved version using ANY.  As before calculate the avg grade for a student and include the student only if average is greater and any of the student averages.  This effectively will include everyone but the person with the lowest average. Compare your answer with the answer from the query using a temporary table.

In [37]:
%%sql
SELECT s.name, avg(e.grade)
FROM Enrollments as e
     JOIN Students as s on e.student_id = s.student_id
GROUP BY s.name
HAVING avg(e.grade) > ANY (SELECT avg(grade)
                            FROM Enrollments as e2 
                           GROUP BY e2.student_id)
ORDER BY s.name









 * postgres://isdb@localhost/lab04
11 rows affected.


name,avg
Allen Thomas,2.5000000000000000
Bill Jones,2.0000000000000000
Bob Dawson,2.0000000000000000
Howard Mansfield,3.0000000000000000
Janet Ladd,4.0000000000000000
Janet Thomas,4.0000000000000000
Joe Adams,4.0000000000000000
John Anderson,4.0000000000000000
Susan Powell,3.0000000000000000
Susan Pugh,2.0000000000000000
